In [113]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import random 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
import pickle
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)

In [ ]:
# Solana: "https://finance.yahoo.com/quote/SOL-USD/news?p=SOL-USD", Key part: "SOL-USD/news?p=SOL-USD"
# Bitcoin: "https://finance.yahoo.com/quote/BTC-USD/news?p=BTC-USD", Key part: "BTC-USD/news?p=BTC-USD"
# Apple: "https://finance.yahoo.com/quote/AAPL/news?p=AAPL", Key part: "AAPL/news?p=AAPL"
# Tesla: "https://finance.yahoo.com/quote/TSLA/news?p=TSLA", Key part: "TSLA/news?p=TSLA"

In [16]:
x = 25
print(f'//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/div/div/ul/li[{x}]/section/div/a/h3')

//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/div/div/ul/li[25]/section/div/a/h3


In [125]:
# company_ticker = 'AAPL'
# url = "https://finance.yahoo.com/quote/" + company_ticker + "/news?p=" + company_ticker
#url = "https://finance.yahoo.com/quote/SOL-USD/news?p=SOL-USD"
url = "https://finance.yahoo.com/quote/AAPL/news"
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
#driver.maximize_window()
driver.implicitly_wait(5)
sleep(3)
driver.get(url)
#Scroll:
sroll_script = "window.scrollTo(0, document.body.scrollHeight);"

# for x in range(5,400,5):
#     try:
#         job_posting = driver.find_element(By.XPATH, f'//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/div/div/ul/li[{x}]/section/div/a')
#         print(job_posting)
#         driver.execute_script("arguments[0].scrollIntoView();",job_posting)
#     except Exception as e:
#         print('element not found: ', e)
#         pass


# //*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/div/div/ul/li[6]/section/div/a

for i in range(50):
    driver.execute_script(scroll_script)
    sleep(0.2)
sleep(2)
for i in range(50):
    driver.execute_script(scroll_script)
sleep(2)
for i in range(20):
    driver.execute_script(scroll_script)
sleep(2)
    
html = driver.page_source
soup_crawl = BeautifulSoup(html, "html.parser")
sleep(3)
# bottom_of_page = driver.find_element(By.XPATH, f'colHeading svelte-1cmifq0')
# driver.execute_script("arguments[0].scrollIntoView();",bottom_of_page)
driver.quit()

In [5]:
article_info = []

for part in soup_crawl.find_all(class_ = "container sz-x-large stream svelte-13zydns responsive"):
    a_tag = part.find('a', class_='subtle-link fin-size-small titles noUnderline svelte-wdkn18')
    #print(a_tag)
    link = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None
    title = a_tag['title'] if a_tag and 'title' in a_tag.attrs else None
    print(title)
    print(link)
    article_info.append([title, link])

links_list = [[elements[0],elements[1]] for elements in article_info if 'https://finance.yahoo.com/news/' in elements[1]]

#links_list

In [8]:
import pickle

file = open('yahoo_sol_article_list2.pkl', 'wb')

# dump information to that file
pickle.dump(links_list, file)

In [117]:
import re
import pandas as pd
import numpy as np
from langchain.llms import OpenAI

# Initialize your LangChain LLM for OpenAI
llm = OpenAI(api_key="xxx")
company = "Apple"
apple_prompt =  (
f"Based on the provided news article about the comapany, {company}, provide a sentiment rating, Y/N answers, category, and an explanation for the ratings/labels in the following format: "
f"'<sentiment score>, <Primarily about {company} Y/N>, <Celebrity endorsement Y/N>, <Technical/security issues Y/N>, "
f"<Product announcement Y/N>, <Likely to affect {company} price soon Y/N>, <Article type Opinion/Analysis/Anouncement/Other>'. "
f"Rate the sentiment on a scale of 1 to 5 regarding the future price of {company} (1=very negative, 5=very positive). "
f"Answer Y for Yes or N for No to the following: Primarily about {company}: Is the article mainly about {company}? Note: This means that {company} is the main subject of this article! If the article is talking about other companies with equal importance, then the label would be a 'No' (N)! If the article is talking about companies to invest in, and {company} is one of them, the label would also be a 'No' (N)! The company, {company}, would need to be the very main subject of the article, and the vast majority of the article would need to be dedicated to discussing {company} for this label to be a Yes (Y). Otherwise, it's a No (N)!!!"
f"Celebrity endorsement: Is there a celebrity endorsement for {company} Note: market analysts/experts do not count."
f"Technical/security issues: Are any technical or security issues mentioned about {company}? Product announcement: Are there any new {company} product announcements? Note products not produced by {company} and not directly tied to {company} do not count!"
f"Likely to affect {company} price soon: Is the information of this article likely to affect the price of {company} within the next day? So if this is a recent development that is likely to cause a spike or a fall of price quickly, then it's a Yes."
f"Article type: Which category would this article fit into the most? Is this just an overall analysis of recent price performance in relation to {company}? Or does the article primary feature opinions of a person or a group of people about {company} and its price? Or is there an important announcement/update about {company}? This includes lawsuits, new product releases, new technological integrations or functionalities, etc. Or is this article 'Other', meaning it doesn't fit into any of the other 3 categories?"
f"I want your answer to be of the following format: 3, Y, Y, N, N, Y, Opinion, explanation. Make sure to include a comma after each rating or label."
)

def find_integer_before_dash(text):
    # Use a regular expression to find digits (\d+) possibly followed by spaces (\s*) and then a dash (-)
    match = re.search(r'(\d+)\s*-', text)
    if match:
        return int(match.group(1))  # Return the integer found before the dash
    else:
        return None  # Return None if no match is found

def analyze_sentiment(text):
    """
    Use LangChain's LLM to analyze sentiment of the given text.
    Returns a sentiment rating between 1 to 5 based on the text.
    """
#    prompt = f"Considering the news article provided, rate the sentiment of the text on a scale of 1 to 5, where 1 signifies very negative sentiment towards the future price of Solana, and 5 signifies very positive sentiment towards the future price of Solana. Provide the rating as a number followed by a brief explanation of the sentiment.\n\nText: {text}"
    prompt = (apple_prompt+ "\n" + f"{text}")

    try:
        response = llm.invoke(prompt)  # Adjust this line based on how you call your API
        # First, split the response to get the scores and explanation separately
        parts = response.split(',', 6)
        # last_rating = parts[5].split(',' )
        # Return all components
        sentiment_rating = re.findall(r'\d+', parts[0])
        #print(sentiment_rating + parts[1:5] + [parts[5][1]] + [parts[5][3:]] + [response])
        return sentiment_rating + parts[1:6] + [parts[6][:4]] + [parts[6][4:]]# + [response]
    except Exception as e:
        print(f"Error processing text: {e}")
        return [None, None, None, None, None, None, None, None]

results = df['text'][20:100].apply(lambda x: analyze_sentiment(x)).apply(pd.Series)

print(results)

# Assign extracted values to new columns in the DataFrame

new_columns = ['sentiment_score', 'about_company', 'celebrity_endorsement', 'technical_issues', 'product_announcement', 'price_effect', 'category', 'explanation']

# Add each new column with a default value of NaN
for column in new_columns:
    df[column] = np.nan

df[new_columns] = results[np.arange(0,len(new_columns))]

print(df)

# Assuming df is your DataFrame
columns_to_check = ['about_company', 'celebrity_endorsement', 'technical_issues', 'product_announcement', 'price_effect']
# Create mask for Y/N columns after stripping spaces
mask_YN = df[columns_to_check].apply(lambda x: x.str.strip().isin(['Y', 'N'])).all(axis=1)
# Create mask for 'sentiment_score' being in the range 1 to 5 (assuming they are stored as strings)
mask_rating = df['sentiment_score'].str.strip().isin(['1', '2', '3', '4', '5'])
mask_category = df['category'].str.strip().isin(['Ana', 'Ann', 'Opi', 'Oth'])
# Combine the masks using logical AND
final_mask = mask_YN & mask_rating & mask_category
# Filter the DataFrame based on the combined mask
df_filtered = df[final_mask]
# Reset the index of the filtered DataFrame
df_filtered = df_filtered.reset_index(drop=True)
# Replace "Y" with 1 and "N" with 0 in the filtered DataFrame
df_filtered[columns_to_check] = df_filtered[columns_to_check].apply(lambda x: x.str.strip().replace({'Y': 1, 'N': 0}))
# Convert 'sentiment_score' to integer in the filtered DataFrame
df_filtered['sentiment_score'] = df_filtered['sentiment_score'].astype(int)

# Display the updated DataFrame
print("Done")


/Users/alex/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Error processing text: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 8258 tokens (8002 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error processing text: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 10038 tokens (9782 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error processing text: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4738 tokens (4482 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Error processing text: Error code: 400 - {'error': {'message': "Th

/var/folders/mq/z3r_z9nd59g6z5rfmlxsltg40000gn/T/ipykernel_15208/2747164482.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan
/var/folders/mq/z3r_z9nd59g6z5rfmlxsltg40000gn/T/ipykernel_15208/2747164482.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan
/var/folders/mq/z3r_z9nd59g6z5rfmlxsltg40000gn/T/ipykernel_15208/2747164482.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [ ]:
jkk = "February 9, 2024 at 7:3"

datetime.strptime(time_element.text[:-4], date_format)

In [81]:
# Open the file for reading
with open('yahoo_sol_article_list2.pkl', 'rb') as file:  # 'rb' means read binary mode
    # Load the list using pickle.load()
    loaded_list = pickle.load(file)

# Now, loaded_list contains the original list
print(loaded_list)

links_list = list(loaded_list)

[['Mark Zuckerberg drags Apple’s new Vision Pro in Instagram video, claiming his Quest 3 headset is the ‘better product, period’', 'https://finance.yahoo.com/news/mark-zuckerberg-drags-apple-vision-143656921.html'], ['1 Super Stock That Could Join Apple, Microsoft, Nvidia, Amazon, Alphabet, and Meta in the $1 Trillion Club Within 10 Years', 'https://finance.yahoo.com/news/1-super-stock-could-join-110000497.html'], ['Zacks Investment Ideas feature highlights: Apple, Deckers Outdoor and Alphabet', 'https://finance.yahoo.com/news/zacks-investment-ideas-feature-highlights-110000743.html'], ['The Best Warren Buffett Stocks to Buy With $3,000 Right Now', 'https://finance.yahoo.com/news/best-warren-buffett-stocks-buy-100400338.html'], ['Apple’s Longest-Serving Designer to Depart Company, Adding to Exodus', 'https://finance.yahoo.com/news/apple-longest-serving-designer-depart-223157021.html'], ['These 3 Companies Are Shattering Quarterly Records', 'https://finance.yahoo.com/news/3-companies-sh

In [83]:
delay = random.uniform(1, 2.2)
articles_text = []
articles_times = []
date_format = "%B %d, %Y at %I:%M %p"
for cnt, url in enumerate(urls):
    sleep(delay)
    if cnt % 8 == 0:
        print('count # -> ',cnt)
        sleep(2.8)
        if len(articles_times) > 0:
            print(articles_times[-1])
    response = requests.get(url)
    if response.status_code == 200:
        article = ''
        link_soup = BeautifulSoup(response.content, 'html.parser')     
        article_content = link_soup.find('div', class_='caas-body')
        time_element = link_soup.find('time')
        datetime_obj = datetime.strptime(time_element.text, date_format) + timedelta(hours=2)
        articles_times.append(str(datetime_obj))
        if article_content:
            # Find all <p> tags within this section
            #print(article_content)
            paragraphs = article_content.find_all('p')
            #print(paragraphs)
            # Print the text of each paragraph
            for paragraph in paragraphs:
                article = article+paragraph.text
            articles_text.append(article)
        else:
            articles_text.append("No article content")
            articles_times.append(None)
    else:
        articles_text.append(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        articles_times.append(None)
        

for link, article, time in zip(links_list, articles_text, articles_times):
    link.append(time)
    link.append(article)

links_list

count # ->  0
count # ->  8
2024-02-13 08:43:00
count # ->  16
2024-02-12 14:22:00
count # ->  24
2024-02-12 04:25:00
count # ->  32
2024-02-10 09:10:00
count # ->  40
2024-02-09 14:04:00
count # ->  48
2024-02-09 05:00:00
count # ->  56
2024-02-08 16:14:00
count # ->  64
2024-02-08 08:45:00
count # ->  72
2024-02-08 05:20:00
count # ->  80
2024-02-07 12:58:00
count # ->  88
2024-02-07 07:08:00
count # ->  96
2024-02-06 23:48:00
count # ->  104
2024-02-06 08:54:00
count # ->  112
2024-02-05 16:16:00
count # ->  120
2024-02-05 09:53:00
count # ->  128
2024-02-05 06:25:00
count # ->  136
2024-02-05 01:43:00
count # ->  144
2024-02-04 06:35:00
count # ->  152
2024-02-03 05:55:00


[['Mark Zuckerberg drags Apple’s new Vision Pro in Instagram video, claiming his Quest 3 headset is the ‘better product, period’',
  'https://finance.yahoo.com/news/mark-zuckerberg-drags-apple-vision-143656921.html',
  '2024-02-14 08:36:00',
  'Mixed martial arts enthusiast Mark Zuckerberg is taking the gloves off, only this time instead of social media rival Elon Musk, he’s gunning for Tim Cook.In a\xa0video post\xa0to his account on Meta’s Instagram, the company founder took his best shot at the Apple CEO’s new Vision Pro, after the mixed reality headset stole the thunder away from Zuckerberg’s $500 Quest 3 launched just\xa0four months prior.“I expected that Quest would be the better value for most people, since it’s really good and it’s seven times less expensive,” the Meta boss said on Tuesday after revealing he just tried out Cook’s $3,500 device.“But after using it, I don’t just think that Quest is the better value—I think that Quest is the better product, period.”Zuckerberg, the

In [126]:
df = pd.DataFrame(links_list)
df.columns = ["headline", "url", "date/time", "text"]


In [101]:
len(df)

160

In [107]:
common_values = pd.Series(list(set(df['headline']) & set(df_old['headline'])))
common_values

0     UPDATE 1-Apple is developing foldable clamshel...
1     Steve Jobs Didn't Think He Needed To Shower Be...
2     Alphabet’s Gemini AI rollout reveals the battl...
3                       Apple's Best Days Are Behind It
4     Apple's Holiday Quarter Broke Years-Long Strea...
                            ...                        
62    Tech News Now: 'Black Mirror' is here, Vision ...
63    Apple is developing foldable clamshell iPhones...
64    2 (More) Artificial Intelligence (AI) Growth S...
65    Why did Tim Cook’s pay package hold up in cour...
66       Here’s Why Bireme Capital Shorted Apple (AAPL)
Length: 67, dtype: object

In [122]:
common_values = pd.Series(list(set(df['headline']) & set(df_old['headline'])))

# Filter out rows with common values in both DataFrames
df = df[~df['headline'].isin(common_values)]
df.reset_index(drop = True, inplace = True)
df

,headline,url,date/time,text
0,Mark Zuckerberg drags Apple’s new Vision Pro i...,https://finance.yahoo.com/news/mark-zuckerberg...,2024-02-14 08:36:00,Mixed martial arts enthusiast Mark Zuckerberg ...
1,"1 Super Stock That Could Join Apple, Microsoft...",https://finance.yahoo.com/news/1-super-stock-c...,2024-02-14 05:00:00,The American economy has a rich history of cre...
2,Zacks Investment Ideas feature highlights: App...,https://finance.yahoo.com/news/zacks-investmen...,2024-02-14 05:00:00,"Chicago, IL – February 14, 2024 – Today, Zacks..."
3,"The Best Warren Buffett Stocks to Buy With $3,...",https://finance.yahoo.com/news/best-warren-buf...,2024-02-14 04:04:00,Got some extra cash you won't be needing anyti...
4,Apple’s Longest-Serving Designer to Depart Com...,https://finance.yahoo.com/news/apple-longest-s...,2024-02-13 18:08:00,(Bloomberg) -- Apple Inc.’s longest-serving se...
5,These 3 Companies Are Shattering Quarterly Rec...,https://finance.yahoo.com/news/3-companies-sha...,2024-02-13 14:34:00,We continue to navigate through earnings seaso...
6,"If You Invested $10,000 in Apple When Tim Cook...",https://finance.yahoo.com/news/invested-10-000...,2024-02-13 09:30:00,Apple (NASDAQ: AAPL) is one of the most influe...
7,11 Stocks That May Be Splitting Soon,https://finance.yahoo.com/news/11-stocks-may-s...,2024-02-13 08:43:00,"In this piece, we will take a look at 11 stock..."
8,Apple's Biggest Threat Isn't What You Think,https://finance.yahoo.com/news/apples-biggest-...,2024-02-13 07:26:00,Even the largest companies on the planet face ...
9,Here’s How Not Holding Apple (AAPL) Affected M...,https://finance.yahoo.com/news/not-holding-app...,2024-02-13 07:00:00,"Mairs & Power, an investment advisor, released..."


In [123]:
file_path = 'apple_headlines_latest.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(file_path, index=False)

In [98]:
len(common_rows)

67

In [100]:
df = df[~df.isin(common_rows).all(axis=1)]

In [89]:
file_path = 'apple_headlines_sentiment_langchain5.xlsx'

# Save the DataFrame to an Excel file
df_old = pd.read_excel(file_path)

In [175]:
import json
with open('sol_soup_yahoo_article1.txt', 'w') as file:
    json.dump(str(link_soup.prettify()), file)

In [176]:
import pickle

filename = 'yahoo_sol_article_list.pkl'

# Open a file for writing
with open(filename, 'wb') as file: 
    pickle.dump(links_list, file)
